In [17]:
import cv2
import numpy as np
import onnxruntime as ort

In [18]:
model_path = r"D:\DS\ViolenceDetection\notebooks\model_quantized.onnx"

In [19]:
video_path=r'D:\DS\ViolenceDetection\notebooks\violentVideo\violentVideo\vid_1.mp4'

In [20]:
def img_preprocessing(img):
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    for i in range(3):
        img[:, :, i] = (img[:, :, i] - mean[i]) / std[i]
    img = np.transpose(img, (2, 0, 1))
    return img

In [21]:
def predict_video(video_path,onnx_model_path):
    #loader model
    session = ort.InferenceSession(onnx_model_path)
    input_name=session.get_inputs()[0].name
    output_name=session.get_outputs()[0].name

    cap = cv2.VideoCapture(video_path)
    fps=cap.get(cv2.CAP_PROP_FPS)
    all_frames=cap.get(cv2.CAP_PROP_FRAME_COUNT)
    X=[]
    y_preds=[]
    frame_interval=int(fps/12)
    for i in range(int(all_frames/frame_interval)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i*frame_interval)
        ret, frame = cap.read()
        cv2.imshow('cammera',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if not ret:
            break
        X.append(img_preprocessing(frame))
    if(len(X)<10):
        print('Error video do not have enough frame for prediction!')
    else:
        for i in range(len(X)-9):
            input_data=np.array([X[i:i+10]],dtype=np.float32)
            # print(input_data.shape)
            outputs = session.run([output_name], {input_name: input_data})
            y_preds.append(np.argmax(outputs[0],axis=1)[0])
    cap.release()
    cv2.destroyAllWindows()
    return y_preds
        

In [22]:
y=predict_video(video_path,model_path)

In [23]:
sum(y)

83